<a href="https://colab.research.google.com/github/Fuenfgeld/TeamDataScDatenmanagementUndArchivierung/blob/main/ProjektCovid19Allergy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Covid und Allergie
Die csv-Dateien sind im Ordner csv_files

Die db-Dateien sind im Ordner db_files

In [75]:
import pandas as pd
import numpy as np
import csv
import sqlite3 as sq
from sqlite3 import Error
from google.colab import drive

drive.mount('/content/drive/', force_remount=True)

csv_path = '/content/drive/MyDrive/csv_files/csv_to_db/'
db_path = '/content/drive/MyDrive/db_files/cov_alle.db'

patient_type = "cov_alle"



Mounted at /content/drive/


In [76]:
sql_table = {} # Tables
sql_index = {} # Indizes
sql_view = {} # Views

# Patients allergy
sql_table["patients_alle"] = """CREATE TABLE IF NOT EXISTS patients_alle (
  Id varchar primary key,
  BIRTHDATE date,
  DEATHDATE date,
  SSN varchar,
  DRIVERS varchar,
  PASSPORT varchar,
  PREFIX varchar,
  FIRST varchar,
  LAST varchar,
  SUFFIX varchar,
  MAIDEN varchar,
  MARITAL varchar,
  RACE varchar,
  ETHNICITY varchar,
  GENDER varchar,
  BIRTHPLACE varchar,
  ADDRESS varchar,
  CITY varchar,
  STATE varchar,
  COUNTY varchar,
  ZIP varchar,
  LAT double,
  LON double,
  HEALTHCARE_EXPENSES varchar,
  HEALTHCARE_COVERAGE varchar
);"""

# Indizes
sql_index["ix_patients_alle_id"] = """CREATE UNIQUE INDEX ix_patients_alle_id on patients_alle(Id);""" 
sql_index["ix_patients_alle_race"] = """CREATE INDEX ix_patients_alle_race on patients_alle(RACE);""" 
sql_index["ix_patients_alle_ethnicity"] = """CREATE INDEX ix_patients_alle_ethnicity on patients_alle(ETHNICITY);"""
sql_index["ix_patients_alle_city"] = """CREATE INDEX ixpatients_alle_city on patients_alle(CITY);"""
sql_index["ix_patients_alle_gender"] = """CREATE INDEX ix_patients_alle_gender on patients_alle(GENDER);"""
# sql_index["ix_patients_alle_country"] = """CREATE INDEX ix_patients_alle_country on patients_alle(COUNTRY);"""
sql_index["ix_patients_alle_zip"] = """CREATE INDEX ix_patients_alle_zip on patients_alle(ZIP);"""

# Patients COVID-19

sql_table["patients_cov"] = """CREATE TABLE IF NOT EXISTS patients_cov (
  Id varchar primary key,
  BIRTHDATE date,
  DEATHDATE date,
  SSN varchar,
  DRIVERS varchar,
  PASSPORT varchar,
  PREFIX varchar,
  FIRST varchar,
  LAST varchar,
  SUFFIX varchar,
  MAIDEN varchar,
  MARITAL varchar,
  RACE varchar,
  ETHNICITY varchar,
  GENDER varchar,
  BIRTHPLACE varchar,
  ADDRESS varchar,
  CITY varchar,
  STATE varchar,
  COUNTY varchar,
  ZIP varchar,
  LAT double,
  LON double,
  HEALTHCARE_EXPENSES varchar,
  HEALTHCARE_COVERAGE varchar
);"""

sql_index["ix_patients_cov_id"] = """CREATE UNIQUE INDEX ix_patients_cov_id on patients_alle(Id);""" 
sql_index["ix_patients_cov_race"] = """CREATE INDEX ix_patients_cov_race on patients_alle(RACE);""" 
sql_index["ix_patients_cov_ethnicity"] = """CREATE INDEX ix_patients_cov_ethnicity on patients_alle(ETHNICITY);"""
sql_index["ix_patients_cov_city"] = """CREATE INDEX ixpatients_cov_city on patients_alle(CITY);"""
sql_index["ix_patients_cov_gender"] = """CREATE INDEX ix_patients_cov_gender on patients_alle(GENDER);"""
# sql_index["ix_patients_cov_country"] = """CREATE INDEX ix_patients_cov_country on patients_alle(COUNTRY);"""
sql_index["ix_patients_cov_zip"] = """CREATE INDEX ix_patients_cov_zip on patients_alle(ZIP);"""

In [77]:
# Encounters allergy

sql_table["encounters_alle"] = """CREATE TABLE IF NOT EXISTS encounters_alle (
                           Id STRING PRIMARY KEY,
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           ORGANIZATIONS STRING,
                           PROVIDER STRING,
                           PAYER STRING,
                           ENCOUNTERCLASS STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           BASE_ENCOUNTER_COST INTEGER,
                           TOTAL_CLAIM_COST INTEGER,
                           PAYER_COVERAGE INTEGER,
                           REASONCODE STRING,
                           REASONDESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients_alle (Id) 
                       );"""
sql_index["ix_encounters_alle_id"] = """CREATE INDEX ix_encounters_alle_id on encounters_alle(Id);""" 
sql_index["ix_encounters_alle_patient"] = """CREATE INDEX ix_encounters_alle_patients on encounters_alle(PATIENT);""" 
sql_index["ix_encounters_alle_description"] = """CREATE INDEX ix_encounters_alle_description on encounters_alle(DESCRIPTION);"""
sql_index["ix_encounters_alle_code"] = """CREATE INDEX ix_encounters_alle_code on encounters_alle(CODE);"""
sql_index["ix_encounters_alle_reasondescription"] = """CREATE INDEX ix_encounters_alle_reasondescription on encounters_alle(REASONDESCRIPTION);"""
sql_index["ix_encounters_alle_reasoncode"] = """CREATE INDEX ix_encounters_alle_reasoncode on encounters_alle(REASONCODE);"""


# Encounters COVID-19

sql_table["encounters_cov"] = """CREATE TABLE IF NOT EXISTS encounters_cov (
                           Id STRING PRIMARY KEY,
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           ORGANIZATIONS STRING,
                           PROVIDER STRING,
                           PAYER STRING,
                           ENCOUNTERCLASS STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           BASE_ENCOUNTER_COST INTEGER,
                           TOTAL_CLAIM_COST INTEGER,
                           PAYER_COVERAGE INTEGER,
                           REASONCODE STRING,
                           REASONDESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients_cov (Id) 
                       );"""

sql_index["ix_encounters_cov_id"] = """CREATE INDEX ix_encounters_cov_id on encounters_cov(Id);""" 
sql_index["ix_encounters_cov_patient"] = """CREATE INDEX ix_encounters_cov_patients on encounters_cov(PATIENT);""" 
sql_index["ix_encounters_cov_description"] = """CREATE INDEX ix_encounters_cov_description on encounters_cov(DESCRIPTION);"""
sql_index["ix_encounters_cov_code"] = """CREATE INDEX ix_encounters_cov_code on encounters_cov(CODE);"""
sql_index["ix_encounters_cov_reasondescription"] = """CREATE INDEX ix_encounters_cov_reasondescription on encounters_cov(REASONDESCRIPTION);"""
sql_index["ix_encounters_cov_reasoncode"] = """CREATE INDEX ix_encounters_cov_reasoncode on encounters_cov(REASONCODE);"""

In [78]:
# Conditions allergy
sql_table["conditions_alle"] = """CREATE TABLE IF NOT EXISTS conditions_alle (
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients_alle (Id) 
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters_alle (Id) 

                       );"""

sql_index["ix_conditions_alle_patient"] = """CREATE INDEX ix_conditions_alle_patient on conditions_alle(PATIENT);"""
sql_index["ix_conditions_alle_encounter"] = """CREATE INDEX ix_conditions_alle_encounter on conditions_alle(ENCOUNTER);"""
sql_index["ix_conditions_alle_code"] = """CREATE INDEX ix_conditions_alle_code on conditions_alle(CODE);"""
sql_index["ix_conditions_alle_description"] = """CREATE INDEX ix_conditions_alle_description on conditions_alle(DESCRIPTION);"""

# Conditions COVID-19
sql_table["conditions_cov"] = """CREATE TABLE IF NOT EXISTS conditions_cov (
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients_cov (Id) 
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters_cov (Id) 

                       );"""

sql_index["ix_conditions_cov_patient"] = """CREATE INDEX ix_conditions_cov_patient on conditions_cov(PATIENT);"""
sql_index["ix_conditions_cov_encounter"] = """CREATE INDEX ix_conditions_acov_encounter on conditions_cov(ENCOUNTER);"""
sql_index["ix_conditions_cov_code"] = """CREATE INDEX ix_conditions_cov_code on conditions_cov(CODE);"""
sql_index["ix_conditions_cov_description"] = """CREATE INDEX ix_conditions_cov_description on conditions_cov(DESCRIPTION);"""

In [79]:
# Procedures allergy

sql_table["procedures_alle"]= """CREATE TABLE IF NOT EXISTS procedures_alle (
                           DATE DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           BASE_COST INTEGER,
                           REASONCODE STRING,
                           REASONDESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients_alle (Id) 
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters_alle (Id) 

                       );"""
sql_index["ix_procedures_alle_patient"] = """CREATE INDEX ix_procedures_alle_patient on procedures_alle(PATIENT);"""
sql_index["ix_procedures_alle_encounter"] = """CREATE INDEX ix_procedures_alle_encounter on procedures_alle(ENCOUNTER);"""
sql_index["ix_procedures_alle_code"] = """CREATE INDEX ix_procedures_alle_code on procedures_alle(CODE);"""
sql_index["ix_procedures_alle_description"] = """CREATE INDEX ix_procedures_alle_description on procedures_alle(DESCRIPTION);"""


# Procedures COVID-19

sql_table["procedures_cov"]= """CREATE TABLE IF NOT EXISTS procedures_cov (
                           DATE DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           BASE_COST INTEGER,
                           REASONCODE STRING,
                           REASONDESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients_cov (Id) 
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters_cov (Id) 

                       );"""
sql_index["ix_procedures_cov_patient"] = """CREATE INDEX ix_procedures_cov_patient on procedures_cov(PATIENT);"""
sql_index["ix_procedures_cov_encounter"] = """CREATE INDEX ix_procedures_cov_encounter on procedures_cov(ENCOUNTER);"""
sql_index["ix_procedures_cov_code"] = """CREATE INDEX ix_procedures_cov_code on procedures_cov(CODE);"""
sql_index["ix_procedures_cov_description"] = """CREATE INDEX ix_procedures_cov_description on procedures_cov(DESCRIPTION);"""

In [80]:
# Observations allergy
sql_table["observations_alle"]= """CREATE TABLE IF NOT EXISTS observations_alle (
                           DATE DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           VALUE STRING,
                           UNITS STRING,
                           TYPE STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients_alle (Id) 
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters_alle (Id) 

                       );"""
sql_index["ix_observations_alle_patient"] = """CREATE INDEX ix_observations_alle_patient on observations_alle(PATIENT);"""
sql_index["ix_observations_alle_encounter"] = """CREATE INDEX ix_observations_alle_encounter on observations_alle(ENCOUNTER);"""
sql_index["ix_observations_alle_code"] = """CREATE INDEX ix_observations_alle_code on observations_alle(CODE);"""
sql_index["ix_observations_alle_description"] = """CREATE INDEX ix_observations_alle_description on observations_alle(DESCRIPTION);"""


# Observations COVID-19
sql_table["observations_cov"]= """CREATE TABLE IF NOT EXISTS observations_cov (
                           DATE DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           VALUE STRING,
                           UNITS STRING,
                           TYPE STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients_cov (Id) 
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters_cov (Id) 

                       );"""
sql_index["ix_observations_cov_patient"] = """CREATE INDEX ix_observations_cov_patient on observations_cov(PATIENT);"""
sql_index["ix_observations_cov_encounter"] = """CREATE INDEX ix_observations_cov_encounter on observations_cov(ENCOUNTER);"""
sql_index["ix_observations_cov_code"] = """CREATE INDEX ix_observations_cov_code on observations_cov(CODE);"""
sql_index["ix_observations_cov_description"] = """CREATE INDEX ix_observations_cov_description on observations_cov(DESCRIPTION);"""

In [81]:
# Views

# Patients
sql_view["v_patients"] = """CREATE VIEW v_patients as
select PA.id PATIENT_Id, BIRTHDATE, DEATHDATE, MARITAL, RACE, ETHNICITY, GENDER, CITY, ZIP, MAX(strftime('%Y', START)) - strftime('%Y', BIRTHDATE) AGE,  'allergy' study 
from patients_alle pa
JOIN encounters_alle EA 
  ON PA.Id = EA.PATIENT
group by PA.id
  union
select PC.id PATIENT_Id, BIRTHDATE, DEATHDATE, MARITAL, RACE, ETHNICITY, GENDER, CITY, ZIP, MAX(strftime('%Y', START)) - strftime('%Y', BIRTHDATE) AGE, 'covid-19' study 
from patients_cov PC 
JOIN encounters_cov EC 
  ON PC.Id = EC.PATIENT
group by PC.id
;"""


# Encounters

sql_view["v_encounters"] = """ CREATE VIEW v_encounters as
select Id ENCOUNTER_Id, "START" , STOP , PATIENT , CODE , DESCRIPTION, BASE_ENCOUNTER_COST , TOTAL_CLAIM_COST , PAYER_COVERAGE , REASONCODE , REASONDESCRIPTION , 'allergy' study from encounters_alle
  union
select Id ENCOUNTER_Id, "START" , STOP , PATIENT , CODE , DESCRIPTION, BASE_ENCOUNTER_COST , TOTAL_CLAIM_COST , PAYER_COVERAGE , REASONCODE , REASONDESCRIPTION , 'covid-19' study from encounters_cov
; """

# Procedures
sql_view["v_procedures"] = """ CREATE VIEW v_procedures as
select DATE, PATIENT, ENCOUNTER, CODE, DESCRIPTION, BASE_COST, REASONCODE, REASONDESCRIPTION, 'allergy' study FROM procedures_alle
  union
select DATE, PATIENT, ENCOUNTER, CODE, DESCRIPTION, BASE_COST, REASONCODE, REASONDESCRIPTION, 'covid-19' study FROM procedures_cov;
"""

# Conditions

sql_view["v_conditions"] = """ CREATE VIEW v_conditions as
select START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION, 'allergy' study FROM conditions_alle
  union
select START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION, 'covid-19' study FROM conditions_cov;
"""

# Observations

sql_view["v_observations"] = """ CREATE VIEW v_observations as
select DATE,PATIENT,ENCOUNTER,CODE,DESCRIPTION,VALUE,UNITS,TYPE, 'allergy' study FROM observations_alle
  union
select DATE,PATIENT,ENCOUNTER,CODE,DESCRIPTION,VALUE,UNITS,TYPE, 'covid-19' study FROM observations_cov;
"""


In [82]:
def connect_to_db(db_file):
    sqlite3_conn = None
    try:
        sqlite3_conn = sq.connect(db_file)
        return sqlite3_conn

    except Error as err:
        print(err)

        if sqlite3_conn is not None:
            sqlite3_conn.close()

In [83]:
def insert_values_to_table(cursor, table_name, csv_file_path):
    # Read CSV file content
    values_to_insert = open_csv_file(csv_file_path)

    # Insert to table
    if len(values_to_insert) > 0:
        column_names, column_numbers = get_column_names_from_db_table(cursor, table_name)

        values_str = '?,' * column_numbers
        # print("*****", values_str, column_names, column_numbers)
        values_str = values_str[:-1]

        sql_query = 'INSERT OR REPLACE INTO ' + table_name + '(' + column_names + ') VALUES (' + values_str + ')'


        cursor.executemany(sql_query, values_to_insert)
        # conn.commit()

        print('SQL insert process finished')
    else:
        print('Nothing to insert')

        # conn.close()

    # else:
        # print('Connection to database failed')

In [84]:
def open_csv_file(csv_file_path):

    with open(csv_file_path, 'r', encoding='utf-8') as csv_file:
        reader = csv.reader(csv_file)
        next(reader)

        data = list()
        for row in reader:
            data.append(row)
        return data

In [85]:
def get_column_names_from_db_table(sql_cursor, table_name):
    table_column_names = 'PRAGMA table_info(' + table_name + ');'
    sql_cursor.execute(table_column_names)
    table_column_names = sql_cursor.fetchall()

    column_count = len(table_column_names)

    column_names = list()

    for name in table_column_names:
        column_names.append(name[1])

    return ', '.join(column_names), column_count

In [86]:
# print(sql_table.keys())
# print(sql_index.keys())
# print(sql_view.keys())

In [87]:
# import csv into db
conn = connect_to_db(db_path)
if conn is not None:
        cursor = conn.cursor()
        for name in sql_table.keys():
          csv_file = csv_path + name + ".csv"
          print(name, csv_file)
          
          cursor.execute(sql_table[name])

          insert_values_to_table(cursor, name, csv_file)
        # conn.commit()

        for ix_name in sql_index.keys():
          cursor.execute(sql_index[ix_name])

        for v_name in sql_view.keys():
          cursor.execute(sql_view[v_name])

        # conn.commit() # Only for permanent changes in der database
else:
        print('Connection to database failed')



patients_alle /content/drive/MyDrive/csv_files/csv_to_db/patients_alle.csv
SQL insert process finished
patients_cov /content/drive/MyDrive/csv_files/csv_to_db/patients_cov.csv
SQL insert process finished
encounters_alle /content/drive/MyDrive/csv_files/csv_to_db/encounters_alle.csv
SQL insert process finished
encounters_cov /content/drive/MyDrive/csv_files/csv_to_db/encounters_cov.csv
SQL insert process finished
conditions_alle /content/drive/MyDrive/csv_files/csv_to_db/conditions_alle.csv
SQL insert process finished
conditions_cov /content/drive/MyDrive/csv_files/csv_to_db/conditions_cov.csv
SQL insert process finished
procedures_alle /content/drive/MyDrive/csv_files/csv_to_db/procedures_alle.csv
SQL insert process finished
procedures_cov /content/drive/MyDrive/csv_files/csv_to_db/procedures_cov.csv
SQL insert process finished
observations_alle /content/drive/MyDrive/csv_files/csv_to_db/observations_alle.csv
SQL insert process finished
observations_cov /content/drive/MyDrive/csv_files

In [88]:
# Star Schema and psoudonym with patients
cursor.executescript("""
  DROP TABLE if exists pseudonym;  
create table pseudonym(
  psid varchar primary key,
  peid varchar not null unique
);

INSERT into pseudonym
  select distinct 'P'||random() psid, PATIENT_Id peid from v_patients vp
    union
  select distinct 'E'||random() psid, ENCOUNTER_Id peid from v_encounters ve
;

drop table if exists dimCity;
create table dimCity(
  cityId integer primary key autoincrement,
  cityName varchar not null unique
);

--select distinct city from v_patients vp where city not like '';
insert into dimCity(cityName)
  select distinct city from v_patients vp where city not like ''; 

drop table if exists dimGender;
create table dimGender(
  genderId integer primary key autoincrement,
  genderName varchar not null unique
);

insert into dimGender(genderName)
  select DISTINCT gender from v_patients vp where gender not like '';

drop table if exists dimEthnicity;
create table dimEthnicity(
  ethnicityId integer primary key autoincrement,
  ethnicityName varchar not null unique
);

insert into dimEthnicity(ethnicityName)
  select DISTINCT ethnicity from v_patients vp where ethnicity not like '';

drop table if exists dimRace;
CREATE table dimRace(
  raceId integer primary key autoincrement,
  raceName varchar not null unique
);

insert into dimRace(raceName)
  select DISTINCT Race from v_patients vp where Race not like '';

drop table if exists dimMarital;
create table dimMarital(
  maritalId integer primary key autoincrement,
  maritalName varchar not null unique
);

insert into dimMarital(maritalName)
  select DISTINCT Marital from v_patients vp where Marital not like '';


drop table if exists dimStudy;
create table dimStudy(
  studyId varchar primary key,
  studyName varchar not null,
  studyDescription varchar
);

insert into dimStudy(studyId, studyName)
  values
  ('covid19', 'Coronavirus SARS-CoV-2'),
  ('allergy', 'Allergic Diseases');


drop table if exists factPatient;
create table factPatient(
  patientId varchar primary key,
  BIRTHYEAR int,
  DEATHYEAR int,
  genderId int,
  raceId int,
  ethnicityId int,
  cityId int,
  maritalId int,
  studyId varchar not null,
    foreign key (genderId)
      references dimGender(genderId),
    foreign key (raceId)
      references dimRace(raceId),
    foreign key (ethnicityId)
      references dimEthnicity(ethnicityId),
    foreign key (cityId)
      references dimCity(cityId),
    foreign key (maritalId)
      references dimMarital(maritalId),
    foreign key (studyId)
      references dimStudy(studyId)
);

insert into factPatient
select 
  psid, 
  cast(substr(BIRTHDATE, 1,4) as integer) BIRTHYEAR, 
  cast(substr(DEATHDATE , 1,4) as integer) DEATHYEAR, 
  dg.genderId,
  dr.raceId , 
  de.ETHNICITYId, 
  dc.CITYId, 
  dm.MARITALid,
  'allergy' studyId 
from patients_alle pa 
JOIN pseudonym p on pa.id = p.peid 
left join dimGender dg on dg.genderName = pa.GENDER 
left join dimRace dr on dr.raceName = pa.RACE
left join dimEthnicity de on de.ETHNICITYname = pa.ETHNICITY
left join dimCity dc on dc.cityName = pa.city
left join dimMarital dm on dm.maritalname = pa.marital
  UNION 
select 
  psid, 
  cast(substr(BIRTHDATE, 1,4) as integer) BIRTHYEAR, 
  cast(substr(DEATHDATE , 1,4) as integer) DEATHYEAR,
  dg.genderId,
  dr.raceId , 
  de.ETHNICITYId, 
  dc.CITYId, 
  dm.MARITALid,
  'covid19' studyId 
from patients_cov pa 
JOIN pseudonym p on pa.id = p.peid 
left join dimGender dg on dg.genderName = pa.GENDER 
left join dimRace dr on dr.raceName = pa.RACE
left join dimEthnicity de on de.ETHNICITYname = pa.ETHNICITY
left join dimCity dc on dc.cityName = pa.city
left join dimMarital dm on dm.maritalname = pa.marital;
     """)
conn.commit()

In [89]:
########################################################################

In [90]:
f_patients = pd.read_sql_query("select * from factPatient;", conn) # first 3 Patients 
f_patients.head(3)

,patientId,BIRTHYEAR,DEATHYEAR,genderId,raceId,ethnicityId,cityId,maritalId,studyId
0,P-1009385298962331527,1985,0,1,1,1,191,1.0,covid19
1,P-1009848095845742503,1989,0,1,1,1,188,1.0,covid19
2,P-1021758004501900891,1946,0,1,1,1,65,1.0,covid19


Patienten

In [91]:
count_patients_cov = pd.read_sql_query("select count(*) patients_covid19 from patients_cov;", conn) #Anzahl an Patienten mit COVID-19
count_patients_cov

,patients_covid19
0,1050


In [92]:
count_patients_alle = pd.read_sql_query("select count(*) patients_allergy from patients_alle;", conn) #Anzahl an Patienten in Allergie
count_patients_alle


,patients_allergy
0,135


In [93]:
#Patienten by Geschlecht in COVID-19
count_patients_gender_cov = pd.read_sql_query("select GENDER, count(id) patients_covid19 from patients_cov group by GENDER;", conn) 
count_patients_gender_cov

,GENDER,patients_covid19
0,F,522
1,M,528


In [94]:
#Patienten by Geschlecht in Allergy
count_patients_gender_alle = pd.read_sql_query("select GENDER, count(id) patients_allergy from patients_alle group by GENDER;", conn) 
count_patients_gender_alle

,GENDER,patients_allergy
0,F,57
1,M,78


In [95]:
#Patienten by Hautfarbe in COVID-19
count_patients_race_cov = pd.read_sql_query("select RACE, count(distinct ID) C_PATIENTS_COVID FROM patients_cov group by RACE;", conn) 
count_patients_race_cov

,RACE,C_PATIENTS_COVID
0,asian,66
1,black,85
2,native,7
3,white,892


In [96]:
#Patienten by Hautfarbe in Allergy
count_patients_race_alle = pd.read_sql_query("select RACE, count(distinct ID) C_PATIENTS_ALLERGY FROM patients_alle group by RACE;", conn) 
count_patients_race_alle

,RACE,C_PATIENTS_ALLERGY
0,asian,3
1,black,21
2,white,111


In [97]:
# Patient und Conditions in conditions_cov

#observations_patients_cov = pd.read_sql_query("select DESCRIPTION, count(distinct PATIENT) C_PATIENTS_covid19 FROM conditions_cov group by DESCRIPTION;", conn) #Anzahl an Patienten mit COVID-19
#observations_patients_cov.head(4)

In [98]:
# Patient und Conditions in conditions_alle

#observations_patients_alle = pd.read_sql_query("select DESCRIPTION, count(distinct PATIENT) C_PATIENTS_allergy FROM conditions_alle group by DESCRIPTION;", conn) 
#observations_patients_alle.head(4)

In [99]:
# Patient und Description in encounter_alle

#encounter_description_patients_alle = pd.read_sql_query("select DESCRIPTION , count(distinct PATIENT) C_PATIENTS_allergy FROM encounters_alle ea  where DESCRIPTION not like '' group by DESCRIPTION ;", conn) 
#encounter_description_patients_alle

In [100]:
# Patient und Description in encounter_cov

#encounter_description_patients_cov = pd.read_sql_query("select DESCRIPTION , count(distinct PATIENT) C_PATIENTS_covid19 FROM encounters_cov co  where DESCRIPTION not like '' group by DESCRIPTION ;", conn) 
#encounter_description_patients_cov

In [101]:
# Patient und Reasondescription in encounter_alle

#encounter_reasondescription_patients_alle = pd.read_sql_query("select REASONDESCRIPTION , count(distinct PATIENT) C_PATIENTS_allergy FROM encounters_alle ea  where REASONDESCRIPTION not like '' group by REASONDESCRIPTION ;", conn) 
#encounter_reasondescription_patients_alle

In [102]:
# Patient und Reasondescription in encounter_cov

#encounter_reasondescription_patients_cov = pd.read_sql_query("select REASONDESCRIPTION , count(distinct PATIENT) C_PATIENTS_covid19 FROM encounters_cov co  where REASONDESCRIPTION not like '' group by REASONDESCRIPTION ;", conn) 
#encounter_reasondescription_patients_cov

In [103]:
# Age analysis (Covid)

ageCov = pd.read_sql_query("""select 
    PA.ID Patient_allergy, 
    MAX(strftime('%Y', START)) - strftime('%Y', BIRTHDATE) AGE_END 
  from patients_cov PA 
  JOIN encounters_cov EA 
    ON PA.Id = EA.PATIENT 
  GROUP BY PA.ID""", conn)

print('Age in Allergy')
print('count ' + str(ageCov['AGE_END'].count()))
print('max ' + str(ageCov['AGE_END'].max()))
print('min ' + str(ageCov['AGE_END'].min()))
print('mean ' + str(ageCov['AGE_END'].mean()))
print('median ' + str(ageCov['AGE_END'].median()))
print('mode')
print(str(ageCov['AGE_END'].mode()))


Age in Allergy
count 1050
max 110
min 1
mean 41.142857142857146
median 39.0
mode
0    22
1    31
dtype: int64


In [104]:
# Age analysis (Allergy)

ageAlle = pd.read_sql_query("""select 
    PA.ID Patient_allergy, 
    MAX(strftime('%Y', START)) - strftime('%Y', BIRTHDATE) AGE_END 
  from patients_alle PA 
  JOIN encounters_alle EA 
    ON PA.Id = EA.PATIENT 
  GROUP BY PA.ID""", conn)
print('Age in Covid')
print('count ' + str(ageAlle['AGE_END'].count()))
print('max ' + str(ageAlle['AGE_END'].max()))
print('min ' + str(ageAlle['AGE_END'].min()))
print('mean ' + str(ageAlle['AGE_END'].mean()))
print('median ' + str(ageAlle['AGE_END'].median()))
print('mode')
print(str(ageAlle['AGE_END'].mode()))

Age in Covid
count 135
max 104
min 0
mean 47.8962962962963
median 49.0
mode
0    18
1    61
2    69
dtype: int64


In [105]:
conn.close()